# Imports for plotting

In [1]:
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np, healpy as hp, h5py, scipy

In [2]:
%config Completer.use_jedi = False

# Load data

In [3]:
with h5py.File('/data/asfe2/Projects/sims/fire/lsr-1-rslice-5.m12f-res7100-md-sliced-gcat-dr2.hdf5', 'r') as hf:
    print(hf.keys())

<KeysViewHDF5 ['A0', 'a_g_bp_val', 'a_g_rp_val', 'a_g_val', 'age', 'alpha', 'b', 'b_true', 'bp_g', 'bp_g_int', 'bp_g_true', 'bp_rp', 'bp_rp_int', 'bp_rp_true', 'calcium', 'carbon', 'dec', 'dec_error', 'dec_true', 'dhel_true', 'dmod_true', 'e_bp_min_rp_val', 'ebv', 'feh', 'g_rp', 'g_rp_int', 'g_rp_true', 'helium', 'l', 'l_true', 'logg', 'lognh', 'lum_val', 'mact', 'magnesium', 'mini', 'mtip', 'neon', 'nitrogen', 'oxygen', 'parallax', 'parallax_error', 'parallax_over_error', 'parallax_true', 'parentid', 'partid', 'phot_bp_mean_mag', 'phot_bp_mean_mag_error', 'phot_bp_mean_mag_int', 'phot_bp_mean_mag_true', 'phot_g_mean_mag', 'phot_g_mean_mag_error', 'phot_g_mean_mag_int', 'phot_g_mean_mag_true', 'phot_rp_mean_mag', 'phot_rp_mean_mag_error', 'phot_rp_mean_mag_int', 'phot_rp_mean_mag_true', 'pmb_true', 'pmdec', 'pmdec_error', 'pmdec_true', 'pml_true', 'pmra', 'pmra_error', 'pmra_true', 'px_true', 'py_true', 'pz_true', 'ra', 'ra_error', 'ra_true', 'radial_velocity', 'radial_velocity_error',

In [4]:
data = {}
keys = ['ra_true', 'dec_true', 'parallax_true', 'pmra_true', 'pmdec_true', 'phot_g_mean_mag_true']
with h5py.File('/data/asfe2/Projects/sims/fire/lsr-1-rslice-5.m12f-res7100-md-sliced-gcat-dr2.hdf5', 'r') as hf:
    for key in keys:
        data[key]=hf[key][...]

# Load Selection Function

In [5]:
from selectionfunctions.config import config
config['data_dir'] = '/data/asfe2/Projects/testselectionfunctions/'

In [6]:
import selectionfunctions.cog_ii
selectionfunctions.cog_ii.fetch()

Checking existing file to see if MD5 sum matches ...
File exists. Not overwriting.
Checking existing file to see if MD5 sum matches ...
File exists. Not overwriting.


In [7]:
import selectionfunctions.cog_ii as CoGII
from selectionfunctions.source import Source
from selectionfunctions.map import coord2healpix

The DR3 selection function is very approximate. It uses the magnitude relation for the DR2 selection function with the DR3 scanning law. We're working on a new DR3 selection function but we won't have it for a while.

In [8]:
dr3_sf = CoGII.dr3_sf(version='modelAB',crowding=True)

Loading auxilliary data ...
Loading selection function ...
Creating selection function interpolator...
t = 5.105 s
  auxilliary:   5.099 s
          sf:   0.002 s
interpolator:   0.004 s
Loading auxilliary data ...


# Apparent magnitude uncertainty

### G-band uncertainty per observation
$G_\mathrm{amp} = \frac{\sqrt{N_G}\sigma_{F_G}}{F_G}$

This is the expected G-band uncertainty per observation. I've taken the median in magnitude bins for all Gaia EDR3 data.

In [9]:
med_gamp = {}
with h5py.File('median_gamp_edr3.h', 'r') as hf:
    for key in ['magbin','med_gamp']: med_gamp[key]=hf[key][...]

In [10]:
gamp_interp = scipy.interpolate.interp1d(med_gamp['magbin']+0.05, med_gamp['med_gamp'], bounds_error=False)

In [11]:
phot_g_error_amp = gamp_interp(data['phot_g_mean_mag_true'])

### Expected number of observations

$N_\mathrm{transit}$ is the expected number of scans across the sky from the scanning law. This is saved inside the dr3 selection function as "_n_field".

I've used the Beta-Binomial distribution to get the expected number of G-band observations.

In [12]:
coords = Source(data['ra_true'], data['dec_true'], unit='deg', frame='icrs', 
                photometry={'gaia_g':data['phot_g_mean_mag_true']})

In [13]:
# Get healpix pixels
hpxidx = coord2healpix(coords.coord, 'icrs', dr3_sf._nside, nest=True)
# Get number of transits from scanning law.
n_transit = dr3_sf._n_field[hpxidx]

In [14]:
# G-observations efficiency
exp_eff = {}
with h5py.File('expected_gobs_efficiency_edr3.h', 'r') as hf:
    for key in ['magbin','mean_eff']: exp_eff[key]=hf[key][...]

In [15]:
# Expected value of Beta-Binomial distribution
eff_interp = scipy.interpolate.interp1d(exp_eff['magbin']+0.05, exp_eff['mean_eff'], bounds_error=False)

In [16]:
# There are 9 CCD observations per transit in Gaia. 
# The efficiency is the expected number of CCD observations which results in a G-band measurement.
phot_g_n_obs = n_transit * 9 * eff_interp(data['phot_g_mean_mag_true'])

### Expected G uncertainty

Invert the G flux amplitude to get back the flux error then use this to resample an observed apparent magnitude.

$\sigma_{F_G} = \frac{G_\mathrm{amp} F_G}{\sqrt{N_G}}$

In [17]:
data['phot_g_mean_flux_true'] = 10**(-data['phot_g_mean_mag_true']/2.5)

In [18]:
data['phot_g_mean_flux_error'] = phot_g_error_amp * data['phot_g_mean_flux_true'] / np.sqrt(phot_g_n_obs)
data['phot_g_mean_flux'] = np.random.normal(data['phot_g_mean_flux_true'], data['phot_g_mean_flux_error'])

In [19]:
data['phot_g_mean_mag'] = -2.5*np.log10(data['phot_g_mean_flux'])

## Apply Selection Function to survey data

In [20]:
coords = Source(data['ra_true'], data['dec_true'], unit='deg', frame='icrs', 
                photometry={'gaia_g':data['phot_g_mean_mag']})

In [21]:
# Evaluate selection probability
data['prob_selection'] = dr3_sf(coords)

In [22]:
# Draw selected sample from Bernoulli distribution
data['selected'] = np.random.rand(len(data['prob_selection']))<data['prob_selection']

# Load Astrometric Spread Function

In [23]:
from scanninglaw.config import config
config['data_dir'] = '/data/asfe2/Projects/testscanninglaw/'

In [24]:
import scanninglaw.asf
scanninglaw.asf.fetch(version='dr3_nominal')

Checking existing file "/data/asfe2/Projects/testscanninglaw/cog/cog_edr3_asf_nominal.h5" to see if MD5 sum matches ...
Checking existing file to see if MD5 sum matches ...


 24.0 MiB of 24.0 MiB |   4.2 MiB/s |###################| 100% | ETA:  00:00:00

In [25]:
import scanninglaw.asf as asf
from scanninglaw.source import Source

In [26]:
# Currently we only have the DR2 ASF. 
# I'm planning to use the EDR3 scanning law to estimate the updated version but a bit short of time just now.
dr3_asf = asf.asf(version='dr3_nominal')

Loading auxilliary data ...
t = 0.015 s
  auxilliary:   0.015 s


## Estimate Covariance

In [27]:
coords = Source(data['ra_true'], data['dec_true'], unit='deg', frame='icrs', 
                photometry={'gaia_g':data['phot_g_mean_mag']})

In [28]:
%time covariance = dr3_asf(coords)

CPU times: user 22.9 s, sys: 11.5 s, total: 34.4 s
Wall time: 11.3 s


In [29]:
pars = ['ra', 'dec', 'parallax', 'pmra', 'pmdec']
for i in range(5):
    for j in range(i+1):
        if i==j: data[pars[i]+'_error'] = np.sqrt(covariance[i,j])
        else: data[pars[j]+'_'+pars[i]+'_corr'] = covariance[i,j]/np.sqrt(covariance[i,i]*covariance[j,j])

## Draw astrometry from uncertainty

In [30]:
r_true = np.array([data[par+'_true'] for par in pars]).T

In [31]:
# Cholesky sampling
chol = np.linalg.cholesky(np.moveaxis(covariance,2,0))
normal_draw = np.random.normal(0,1,size=(r_true.shape[0], 5))
r_observed = (chol@normal_draw[:,:,None])[:,:,0]

In [32]:
for i, par in enumerate(pars):
    data[par] = r_observed[:,i]

In [33]:
data

{'ra_true': array([283.83672593, 284.00974235, 277.0753315 , ..., 291.76433796,
        285.62463018, 284.35179703]),
 'dec_true': array([ -9.93852985,  -8.02840473,  -9.10765778, ..., -29.72356809,
        -30.20804083, -32.92743411]),
 'parallax_true': array([0.13658888, 0.13663149, 0.13693351, ..., 0.13636245, 0.13676623,
        0.1346569 ]),
 'pmra_true': array([ -1.72112848,  -1.84009815,  -2.50622303, ..., -10.1635837 ,
          1.13589567,   1.92823289]),
 'pmdec_true': array([-2.69772654, -3.05074041, -5.19629999, ..., -8.99596849,
         0.23870632, -6.22867753]),
 'phot_g_mean_mag_true': array([20.271832, 19.74104 , 19.719778, ..., 17.819677, 20.877844,
        19.187195], dtype=float32),
 'phot_g_mean_flux_true': array([7.7851645e-09, 1.2693585e-08, 1.2944602e-08, ..., 7.4495318e-08,
        4.4551554e-09, 2.1140822e-08], dtype=float32),
 'phot_g_mean_flux_error': array([6.56448250e-11, 6.10510410e-11, 5.40214643e-11, ...,
        9.78550969e-11, 4.54193352e-11, 7.368504